# AI2023_C6_Project: Distribusi Strategis Tangki Air
Ketua: 
- Hugo Alfedo Putra (225150201111013)

Anggota:
- Farrel Rakha Dzakwan (NIM)
- Kartika Madania (NIM)
- Nafakhatul Fadhliyah (NIM)
- Rayhan Egar Sadtya N. (225150201111014)

## Project Description

- Detailed proposal: <a href="https://drive.google.com/file/d/17-KTcC_7lHK3ccReEwo3SYMl73sKyIs8/view?usp=sharing">Click Here (UB Account)</a>
- Case example: <a href="https://drive.google.com/file/d/1cw2pzo-zphH3vdL3rC1ZDHoAic22058L/view?usp=sharing">Click Here (UB Account)</a>

## Dependencies

In [192]:
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt

## Class: Node
Node mereperesentasikan sebuah desa, memiliki atribut seperti:
- (str) name 
- (int[]) coordinate
- (int) population
- (object[]) neighbors

In [193]:
class Node:
    
    # initialization function
    def __init__(self, name, coordinate_x, coordinate_y, population):
        self.name = name
        self.coordinate = np.array([coordinate_x, coordinate_y])
        self.population = population
        self.neighbors = []
    
    # Establishes bi-directional path
    def add_neighbor(self, neighbor):
        # The if-condition below guarantees no recursive connections
        # i.e. a node can't be neighbors with itself
        if self.name != neighbor.name:
            sld = np.sqrt(np.power(neighbor.coordinate[0] - self.coordinate[0], 2) + np.power(neighbor.coordinate[1] - self.coordinate[1] ,2))
            cost = random.choice([i for i in range(100)])
            self.neighbors.append([neighbor, sld, cost])
            neighbor.neighbors.append([self, sld, cost])
    
    # Get neighbors
    def get_neighbor(self):
        its_neighbors = []
        for i in range(len(self.neighbors)):
            its_neighbors.append(self.neighbors[i][0].name)
        return its_neighbors
    
    def get_cost(self):
        its_costs = []
        for i in range(len(self.neighbors)):
            its_costs.append(self.neighbors[i][2])
        return its_costs
    
    def get_sld(self):
        its_slds = []
        for i in range(len(self.neighbors)):
            its_slds.append(self.neighbors[i][1])
        return its_slds

    # Print node information
    def print_info(self):
        print(f"Name: {self.name}")
        print(f"Coordinate [X, Y]: {self.coordinate}")
        print(f"Population: {self.population}")
        print(f"Neighbors: {self.get_neighbor()}")
        print(f"Cost: {self.get_cost()}")
        print(f"SLD: {self.get_sld()}")

    # Quick debug purposes
    def minimal_print(self):
        print(f"ID: {self.name} \t -> {self.get_neighbor()}")

## Class: Environment

In [194]:
class Environment:
    
    # Initialization function
    def __init__(self, *args):

        if len(args) > 1 and isinstance(args[0], str):
            # Random generation needs name, area_size, node_number
            self.name = args[0]
            self.node_number = args[2]
            self.empty_node_avail = args[2] # kind of redundant (read more below)
            self.area_size = args[1]
            self.area = [[None for _ in range(args[1])] for _ in range(args[1])]

            self.node_list = [] # Used to store generated nodes
            # Note: making an array filled with None and then *appending* instead of *updating*
            # the values doesn't make any sense because you'd have Nones in front of everything else
            
            self.node_score = [None for _ in range(args[2])] # Used to store f(n) values for generated nodes

        # Nodes generated from data will only need Node
        if len(args) == 1 and isinstance(args[0], list):
            self.nodes = args[0]
            self.node_number = len(args[0])
            self.node_list = [] 
            self.node_score = [None for _ in range(len(args[0]))]

    # Add node
    def add_node(self, node):
        self.area[node.coordinate[0]][node.coordinate[1]] = node
        self.node_list.append(node)
        self.empty_node_avail -= 1 # you have a while loop using "remaining" as a decreasing variable, I don't think this is useful
    
    def add_node_from_data(self, node):
        self.node_list.append(node)

    # Generate test case environment
    def test_case(self):
        O_node = Node("O", 0, 0, 0)
        A_node = Node("A", -3, 3, 300)
        B_node = Node("B", 1, 6, 100)
        C_node = Node("C", 5, 5, 150)
        D_node = Node("D", 2, 3, 400)
        E_node = Node("E", 5, 3, 250)
        F_node = Node("F", 4, 0, 75)
        self.add_node(O_node)
        self.add_node(A_node)
        self.add_node(B_node)
        self.add_node(C_node)
        self.add_node(D_node)
        self.add_node(E_node)
        self.add_node(F_node)
        O_node.add_neighbor(D_node)
        O_node.add_neighbor(F_node)
        A_node.add_neighbor(B_node)
        A_node.add_neighbor(D_node)
        B_node.add_neighbor(A_node)
        B_node.add_neighbor(D_node)
        B_node.add_neighbor(C_node)
        C_node.add_neighbor(B_node)
        C_node.add_neighbor(E_node)
        D_node.add_neighbor(E_node)
        D_node.add_neighbor(F_node)
        self.node_number = 7

    # Populate area 
    def populate_area(self):
        # Makes sure the origin node is registered in node_list and area after creation
        new_node = Node("origin", 0, 0, 0)
        self.add_node(new_node)

        remaining = self.node_number
        naming_counter = 0

        # Generate randomly-placed nodes
        while(remaining > 0):

            # Randomize index
            x_loc = np.random.randint(1, self.area_size)
            y_loc = np.random.randint(1, self.area_size)

            # If index empty, initialize node
            if (self.area[x_loc][y_loc] is None):
                # Creates data for new_node
                node_name = str(self.name) + "node" + str(naming_counter) 
                naming_counter += 1
                node_population = np.random.randint(50, 1000)

                # Creates the new_node
                new_node = Node(node_name, x_loc, y_loc, node_population)

                # Adds the node using a function *you* have created before
                self.add_node(new_node)

                remaining -= 1

        # Generate randomly-put edges (neighbors) between nodes
        # Constraint: 
        #   1. Guaranteed edge from "origin" to at least one node
        #   2. Every node is reachable

        # choice is a list containing all possible integers from 0 to self.node_number
        choice = [i for i in range(self.node_number)]

        for i in range(len(self.node_list)-1):
            # Makes sure there is a path from origin to the last node where
            # it visits every other nodes on the way (satisfies all constraints)
            self.node_list[i].add_neighbor(self.node_list[i+1])

            # Connects one node with random neighbors if it passes a certain
            # threshold (chance > k)
            chance = np.random.random()
            if chance > 0.69: # change this if you want
                self.node_list[i].add_neighbor(self.node_list[random.choice(choice)])

    def populate_from_data(self):
        # Pick a place with the heighest population count as the origin
        max_pop = 0; idx = 0
        for i in range(len(self.nodes)):
            # Adds all the nodes anyway; two birds with one stone
            self.add_node_from_data(self.nodes[i])

            # Finds the place with the highest population count and gets its index
            if self.nodes[i].population > max_pop:
                max_pop = self.nodes[i].population
                idx = i
        
        new_node = Node(self.nodes[idx].name, self.nodes[idx].coordinate[0], self.nodes[idx].coordinate[1], self.nodes[idx].population)
        self.add_node_from_data(new_node)

        # Add random neighbors to each other 
        # Generate randomly-put edges (neighbors) between nodes
        # Constraint: 
        #   1. Guaranteed edge from "origin" to at least one node
        #   2. Every node is reachable

        # choice is a list containing all possible integers from 0 to self.node_number
        choice = [i for i in range(len(self.nodes))]

        for i in range(len(self.nodes)-1):
            if i != idx:
                # Makes sure there is a path from origin to the last node where
                # it visits every other nodes on the way (satisfies all constraints)
                self.nodes[i].add_neighbor(self.nodes[i+1])

                # Connects one node with random neighbors if it passes a certain
                # threshold (chance > k)
                chance = np.random.random()
                if chance > 0.69: # change this if you want
                    self.nodes[i].add_neighbor(self.nodes[random.choice(choice)])

### Environment Use Case

In [195]:
env = Environment("test", 16, 16)
env.test_case()
for i in range(env.node_number):
    env.node_list[i].minimal_print()

ID: O 	 -> ['D', 'F']
ID: A 	 -> ['B', 'D', 'B']
ID: B 	 -> ['A', 'A', 'D', 'C', 'C']
ID: C 	 -> ['B', 'B', 'E']
ID: D 	 -> ['O', 'A', 'B', 'E', 'F']
ID: E 	 -> ['C', 'D']
ID: F 	 -> ['O', 'D']


### Referensi Geolokasi Database

Teks di bawah ini sekadar mencontohkan cara kerja kelompok kami:

Didapatkan sebuah batasan berupa koordinat-koordinat dari website https://boundingbox.klokantech.com sebagai berikut:
- batasan Barat: `112.471417`
- batasan Selatan: `-8.048836`
- batasan Timur: `112.745595`
- batasan Utara: `-7.80712`

dengan visualisasi:
<br>
<img src="./visualisasi_1.png" width="800">

dan juga database `ref_db.csv` yang akan digunakan untuk menentukan desa-desa mana saja yang berada di dalam batasan-batasan tersebut.


In [226]:
batasan = [115.1402,-8.7033,115.2088,-8.6458]
# dengan urutan barat, selatan, timur, utara

In [227]:
ref_db = pd.read_csv('ref_db.csv')
ref_db.head()

,name,lat,lng,pop
0,LATIUNG,2.357417,96.452420,779
1,LABUHAN BAJAU,2.391348,96.479117,88
2,SUAK LAMATAN,2.353943,96.364809,2205
3,ANA AO,2.439100,96.450882,1890
4,LATALING,2.458705,96.455400,1577


In [228]:
def cari_desa(batasan):
    kumpulan_desa = []
    for i in range(len(ref_db)):
        if ref_db['lat'][i] >= batasan[1] and ref_db['lat'][i] <= batasan[3] and ref_db['lng'][i] >= batasan[0] and ref_db['lng'][i] <= batasan[2] and ref_db['lat'][i] != 0 and ref_db['lng'][i] != 0:
            new_node = Node(ref_db['name'][i],ref_db['lat'][i],ref_db['lng'][i],ref_db['pop'][i])
            kumpulan_desa.append(new_node)
        else: continue
    return kumpulan_desa

In [229]:
kumpulan_desa = cari_desa(batasan)
print(len(kumpulan_desa))

9


In [230]:
env_on_data = Environment(kumpulan_desa)
env_on_data.populate_from_data()
for i in range(env_on_data.node_number):
    env_on_data.node_list[i].minimal_print()

ID: LEGIAN 	 -> ['SEMINYAK', 'KEROBOKAN']
ID: SEMINYAK 	 -> ['LEGIAN', 'KEROBOKAN KELOD']
ID: KEROBOKAN KELOD 	 -> ['SEMINYAK', 'KEROBOKAN', 'TEGAL KERTHA']
ID: KEROBOKAN 	 -> ['LEGIAN', 'KEROBOKAN KELOD', 'DAUH PURI KAUH']
ID: DAUH PURI KAUH 	 -> ['KEROBOKAN', 'PEMECUTAN']
ID: PEMECUTAN 	 -> ['DAUH PURI KAUH', 'TEGAL HARUM']
ID: TEGAL HARUM 	 -> ['PEMECUTAN']
ID: TEGAL KERTHA 	 -> ['KEROBOKAN KELOD', 'PADANGSAMBIAN']
ID: PADANGSAMBIAN 	 -> ['TEGAL KERTHA']


## Environment Evaluation

A* Search, expand minimum $f(n) = g(n) + h(n)$ node first.

$$ g(n) = cost(root, n) + \frac{\sum _{i = 0}^n cost(i, n)}{n-1} $$

$$ h(n) = \frac{\sum _{i=0}^n h_{SLD}(i)}{n-1} + \frac{\sum _{i = 0}^{n-1}\text{population}(h) * d_{SLD}(i, k)}{\sum _{i=0}^n \text{population}(h)} $$

$$ f(n) = cost(root, n) + \frac{\sum _{i = 0}^n cost(i, n)}{n-1} + \frac{\sum _{i=0}^n h_{SLD}(i)}{n-1} + \frac{\sum _{i = 0}^{n-1}\text{population}(h) * d_{SLD}(i, k)}{\sum _{i=0}^n \text{population}(h)} $$